In [ ]:
from tensorflow.keras.models import load_model
import cv2
import numpy  as np

model = load_model('traffic_sign_model.keras')
img = cv2.imread('D:\\archive\\Meta\\1.png')  # Загружаем новое изображение
img = cv2.resize(img, (64, 64)) / 255.0  # Предобработка
pred = model.predict(np.expand_dims(img, axis=0))
pred_class = np.argmax(pred)
print(f"Предсказанный класс: {pred_class}")

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)